In [1]:
import pandas as pd
import requests
import json
import pubchempy as pcp


In [35]:
#testing with retrieving ghs classifications for choline chloride (cid = 6209)
safety_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/6209/JSON?heading=GHS+Classification"

request = requests.get(safety_url)
request_json = request.json()

In [118]:
GHS_information_list = []
for i in range(len(request_json['Record']['Section'][0]['Section'][0]['Section'][0]['Information'][2]['Value']['StringWithMarkup'])):
    temp_list = []
    temp_list.append(request_json['Record']['Section'][0]['Section'][0]['Section'][0]['Information'][2]['Value']['StringWithMarkup'][i]['String'])
    GHS_information_list.append(temp_list)

In [119]:
GHS_information_list

[['Aggregated GHS information provided by 484 companies from  14  notifications to the ECHA C&L Inventory. Each notification may be associated with multiple companies.'],
 ['Reported as not meeting GHS hazard criteria by 194 of 484 companies. For more detailed information, please visit  ECHA C&L website'],
 ['Of the 12 notification(s) provided by 290 of 484 companies with hazard statement code(s):'],
 ['H315 (99.31%): Causes skin irritation [Warning Skin corrosion/irritation]'],
 ['H319 (99.31%): Causes serious eye irritation [Warning Serious eye damage/eye irritation]'],
 ['H335 (98.62%): May cause respiratory irritation [Warning Specific target organ toxicity, single exposure; Respiratory tract irritation]'],
 ['Information may vary between notifications depending on impurities, additives, and other factors. The percentage value in parenthesis indicates the notified classification ratio from companies that provide hazard codes. Only hazard codes with percentage values above 10% are s

In [133]:
hazard_description_list = []
for item in GHS_information_list:
    res = [idx for idx in item if idx[0] == 'H']
    hazard_description_list.append(res)
    for item in hazard_description_list:
        if len(item) == 0:
            hazard_description_list.remove(item)
        


In [134]:
hazard_description_list

[['H315 (99.31%): Causes skin irritation [Warning Skin corrosion/irritation]'],
 ['H319 (99.31%): Causes serious eye irritation [Warning Serious eye damage/eye irritation]'],
 ['H335 (98.62%): May cause respiratory irritation [Warning Specific target organ toxicity, single exposure; Respiratory tract irritation]']]